In [ ]:
%reset -f

In [ ]:
%pwd

In [ ]:
%%javascript
document.addEventListener('keydown', function(e) {
  if (e.key === 'F12') {
    e.preventDefault();
    if (document.fullscreenElement) {
      document.exitFullscreen();
    } else {
      document.documentElement.requestFullscreen();
    }
  }
});


In [3]:
import arcpy
import os

myProject = 'MyProject1'
dataDIR = r'C:\Users\fullh\Documents\ArcGIS\Projects\{}\Data'.format(myProject)
gdbDIR = r'C:\Users\fullh\Documents\ArcGIS\Projects\{}\{}.gdb'.format(myProject, myProject)
arcpy.env.overwriteOutput = True
arcpy.env.workspace = dataDIR

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.ia.RasterCalculator(
    rasters=['CCSM4_1974_2013_ero.tif'],
    input_names=['raster'],
    expression='raster >= 0'
  )
output_raster.save(os.path.join(gdbDIR, 'clip_raster'))

with arcpy.EnvManager(extent="MAXOF"):
  arcpy.management.CompositeBands(
    in_rasters="RAP_treNPP.tif;RAP_shrNPP.tif;RAP_pfgNPP.tif;RAP_ltr.tif;RAP_bgr.tif;RAP_afgNPP.tif;POL_ph.tif;POL_sand.tif;POL_om.tif;POL_ksat.tif;POL_clay.tif;POL_bd.tif;pmomx_1974_2013.tif;pmomn_1974_2013.tif;DEM_Slope.tif;DEM_Elev.tif;CCSM4_1974_2013_swe.tif;CCSM4_1974_2013_pet.tif;CCSM4_1974_2013_ndays.tif;CCSM4_1974_2013_ero.tif;CCSM4_1974_2013_consec.tif;tmean_1974_2013.tif;accm_1974_2013.tif",
    out_raster=os.path.join(gdbDIR, "CompositeBands")
  )

arcpy.env.workspace = gdbDIR

with arcpy.EnvManager(outputZFlag="Disabled", outputMFlag="Disabled"):
  arcpy.conversion.RasterToPolygon(
    in_raster='clip_raster',
    out_polygon_features=os.path.join(gdbDIR, 'clip_shape'),
    simplify="SIMPLIFY",
    raster_field="Value",
    create_multipart_features="SINGLE_OUTER_PART",
    max_vertices_per_feature=None
  )

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.ia.RasterCalculator(
    rasters=['CompositeBands'],
    input_names=['raster'],
    expression='raster * 1'
  )
output_raster.save(os.path.join(gdbDIR, 'NoData_Composite'))

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.ia.RasterCalculator(
    rasters=['NoData_Composite'],
    input_names=['raster'],
    expression='Con(IsNull(raster),1,raster)'
  )
output_raster.save(os.path.join(gdbDIR, 'Filled_Composite'))

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.sa.ExtractByMask(
    in_raster='Filled_Composite',
    in_mask_data='clip_shape',
    extraction_area="INSIDE",
    analysis_extent='-120.004083333317 31.3333333333001 -102.995916734683 42.004083290617 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
  )
output_raster.save(os.path.join(gdbDIR, 'Filled_Extract'))
